# This notebook is used for modelling between the livability of melbourne and rental affordability

For the sake of simplicty, this notebook contains the following
- Livability scores of melbourne
- Rental affordability of greater Melbourne

In [2]:
# Load libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from statsmodels.stats.api import anova_lm

## Data processing

The below data will be used for this model:

### Melbourne livability score

- This dataset contains multiple liveability related scores for multiple regions

### Predictor variable: Greater Melbourne rental affordability index

- This dataset contains information about the rental affordability 

## Initial data analysis

In [3]:
# Load data

liveability = pd.read_csv("../data/predictor/Melbourne Liveability Indicators.csv")
rental_affordability = pd.read_csv("../data/response/rental_affordability.csv")
rental_affordability.columns = rental_affordability.columns.str.replace(' ', '')

FileNotFoundError: [Errno 2] No such file or directory: '../data/predictor/Melbourne Liveability Indicators.csv'

In [ ]:
liveability.columns.sort_values()

In [ ]:
set(liveability['period'])

For the liveability dataset, we have information from 2006, 2011~2019. Now we proceed to look at rental_affordability

In [ ]:
rental_affordability

In [ ]:
rental_affordability.columns.sort_values()

From a quick look , it looks like geography_name=3005 seems to only hold NaN values, and thus can get rid of it

In [ ]:
rental_affordability[rental_affordability['geography_name']==3005]

In [ ]:
rental_affordability[['unique_id', 'city', 'geography_name', 'state']]

From the above information, we can remove all columns but geography_name (which seems to be postcode), and also remove where geography_name=3005

In [ ]:
rental_affordability = rental_affordability.drop(3)
rental_affordability = rental_affordability.reset_index()
rental_affordability.drop(columns=rental_affordability.columns[0], axis=1, inplace=True)

## Dataset cleanup
Now that we have the appropriate information, we finalise the dataset so that we can use for modelliing

In [4]:
liveability

NameError: name 'liveability' is not defined

In [5]:
rental_affordability

NameError: name 'rental_affordability' is not defined

Firstly, we need to be able to condense both of the datasets into year so that we can model easier. For the liveability score, we'll combine numerator + denominator for a whole year, then calculate a new percentage for said year. Further, we only want percentages for our dataset since counts will not give us any relevant predictions, as we do not need to create a model but rather see some correlation between the 2 datasets

In [126]:
# Get the subset of the dataframe with the data we want

liveability = liveability[liveability['type'] == 'Liveability']
liveability = liveability[liveability['value_type'] == 'Percentage']
liveability['period'] = liveability['period'].str.extract(r'(\d{4})')
set(liveability['topic'])

{'Economy',
 'Finance',
 'Governance',
 'Profile - housing, government and economy',
 'Profile - people',
 'Transportation'}

In [127]:
# Then remove all NaN values

liveability = liveability.dropna(subset=['numerator', 'denominator'])

In [132]:
# Turn all numerical columns into int for calculation

liveability['numerator'] = liveability['numerator'].astype(str).str.replace(',', '').astype(float).astype(int)
liveability['denominator'] = liveability['denominator'].astype(str).str.replace(',', '').astype(float).astype(int)

In [133]:
liveability[liveability['indicator'] == 'Percentage of country GDP']

,type,topic,id,indicator,period,numerator,denominator,value,value_type,sources
46,Liveability,"Profile - housing, government and economy",PROH_8,Percentage of country GDP,2016,1323421072479,73043232000,5.52,Percentage,"1. The World Bank data, GDP (current US$) for ..."
47,Liveability,"Profile - housing, government and economy",PROH_8,Percentage of country GDP,2015,1208039015868,68408312000,5.66,Percentage,"1. The World Bank data, GDP (current US$) for ..."
48,Liveability,"Profile - housing, government and economy",PROH_8,Percentage of country GDP,2013,1464955475993,83367000000,5.69,Percentage,"1. The World Bank data, GDP (current US$) for ..."
247,Liveability,"Profile - housing, government and economy",PROH_8,Percentage of country GDP,2014,1349034029453,69580800000,5.16,Percentage,"1. The World Bank data, GDP (current US$) for ..."


After a qucik look, for some reason the numerator & denominator for all GDP related rows are swapped. Further, even if we did swap the columns around for appropriate values, the numbers are much greater than the other values used for calculation, thus adding these values in will skew our data so that the majority of the percentage calculation would come from these 2 rows. Thus, opt to remove these rows

In [134]:
liveability = liveability[liveability['indicator'] != 'Percentage of country GDP']

In [135]:
liveability[liveability['period'] == '2014']

,type,topic,id,indicator,period,numerator,denominator,value,value_type,sources
4,Liveability,Economy,ECO_4,Percentage of persons (city population) in ful...,2014,55359,127975,43.26,Percentage,"Geografia, Relative Poverty and Employment Mea..."
5,Liveability,Economy,ECO_5,Youth unemployment rate,2014,1367,13048,10.48,Percentage,"Geografia, Relative Poverty and Employment Mea..."
12,Liveability,Finance,FIN_1,Debt service ratio (debt service expenditure a...,2014,0,378466000,0.00,Percentage,1-3. City of Melbourne Annual Report 2017-18; ...
15,Liveability,Finance,FIN_2,Capital spending as a percentage of total expe...,2014,82078000,457698000,17.93,Percentage,1-3. City of Melbourne Annual Report 2017-18; ...
18,Liveability,Finance,FIN_4,Tax collected as a percentage of tax billed,2014,228981000,226920000,100.91,Percentage,1-3. City of Melbourne Annual Report 2017-18; ...
30,Liveability,Governance,GOV_3,Percentage of women employed in the city gover...,2014,1019,2631,38.73,Percentage,1-3. City of Melbourne Annual Report 2017-18; ...
51,Liveability,Profile - people,PROP_10,Percentage of population that are adults (25-6...,2014,74084,127975,57.89,Percentage,"1. Australian Bureau of Statistics, 3235.0 Reg..."
143,Liveability,Profile - people,PROP_12,Population dependency ratio (people aged under...,2014,13,86,15.52,Percentage,"1. Australian Bureau of Statistics, 3235.0 Reg..."
146,Liveability,Profile - people,PROP_5,Percentage of population that are seniors (65+),2014,8413,127975,6.57,Percentage,"Australian Bureau of Statistics, Census of Pop..."
202,Liveability,Economy,ECO_3,Percentage of city population living in poverty,2014,11556,127975,9.03,Percentage,"Geografia, Relative Poverty and Employment Mea..."


In [165]:
liveability_clean = liveability[['period', 'numerator', 'denominator']].groupby('period').sum().reset_index()
liveability_clean['percentage'] = ((liveability_clean['numerator'] * 100)/liveability_clean['denominator']).round(2)
liveability_clean.rename(columns={'period': 'year'}, inplace=True)

In [166]:
liveability_final = liveability_clean[['year', 'percentage']]

Note that for this liveability scoring dataset, it doesn't make much sense to add the differing factors to simply calculate a new 'percentage' per year. However, since the goal of this analysis is to see if there a correlation between the 2 datasets, so we ignore this for the sake of simplicity and understanding :)

Now we clean up the dataset for rental affordability

In [138]:
rental_affordability

,geography_name,rai_cityadjusted_total_2011_q2,rai_cityadjusted_total_2011_q3,rai_cityadjusted_total_2020_q1,rai_cityadjusted_total_2011_q1,state,rai_cityadjusted_total_2020_q3,rai_cityadjusted_total_2019_q4,rai_cityadjusted_total_2020_q2,rai_cityadjusted_total_2011_q4,...,rai_cityadjusted_total_2012_q1,rai_cityadjusted_total_2015_q3,rai_cityadjusted_total_2012_q4,rai_cityadjusted_total_2014_q3,rai_cityadjusted_total_2012_q3,rai_cityadjusted_total_2014_q4,rai_cityadjusted_total_2015_q1,rai_cityadjusted_total_2014_q1,rai_cityadjusted_total_2014_q2,rai_cityadjusted_total_2015_q4
0,3000,98.33916,98.33916,109.61540,93.91771,VIC,144.2308,109.61540,137.0192,100.62610,...,97.23423,102.56410,103.71650,107.33450,96.15385,107.33450,102.79250,102.56410,102.56410,114.04290
1,3003,97.23423,100.62610,107.46610,88.75740,VIC,144.2308,109.61540,121.7949,81.64006,...,94.06355,103.71650,100.33440,107.33450,90.14423,102.56410,102.56410,113.96010,99.25558,119.60600
2,3004,89.21491,88.30455,103.41070,84.13462,VIC,130.3781,105.39940,111.8524,99.46950,...,86.53846,96.15385,94.19152,98.19967,90.14423,97.16599,96.15385,102.56410,97.16599,102.16350
3,3006,81.64006,78.67133,99.65035,77.66272,VIC,144.2308,99.65035,121.7949,83.21006,...,81.64006,87.91209,88.75740,88.75740,81.64006,90.49774,87.08273,90.49774,90.49774,98.07692
4,3008,77.26648,83.21006,95.31773,74.78632,VIC,135.7466,97.87088,105.3994,79.39308,...,82.41758,90.49774,92.30769,89.61912,88.30455,92.30769,90.49774,93.24009,94.19152,96.15385


In [142]:
# Extract year from column names with year, and extract metadata information 
year_columns = rental_affordability.columns.str.extract(r'(\d{4})')[0]

# If the above pattern was matched, then it will be the year (e.g. 2012), but if it wasn't matched, then it returns NaN
pattern_columns = rental_affordability.columns[year_columns.notna()]
metadata_columns = rental_affordability.columns[year_columns.isna()]

In [144]:
rental_affordability_clean = pd.DataFrame()
unique_years = sorted(year_columns.dropna().unique())

for year in unique_years:
    # Select columns for the current year
    year_cols = rental_affordability.columns[year_columns == year]
    # Sum values across these columns
    rental_affordability_clean[year] = rental_affordability[year_cols].sum(axis=1)

In [154]:
rental_years = []

for i in range(2011, 2022):
    rental_years.append(str(i))

In [162]:
rental_affordability_staging = pd.concat([rental_affordability[metadata_columns], rental_affordability_clean], axis=1)
rental_affordability_staging = rental_affordability_staging[rental_years]

In [163]:
# Since we want to look at all of greater Melbourne, we collapse the dataset and add all the scorings by year
year_sums = []

for year in rental_years:
    year_sums.append(rental_affordability_staging[year].sum().round(2))

In [167]:
rental_affordability_final = pd.DataFrame()
rental_affordability_final['year'] = rental_years
rental_affordability_final['rai'] = year_sums

Thus, here are the 2 final datasets we will use for our analysis:

In [171]:
liveability_final

,year,percentage
0,2006,51.20
1,2011,52.91
2,2012,89.50
3,2013,9.49
4,2014,45.99
5,2015,47.51
6,2016,48.43
7,2017,47.85
8,2018,45.37


In [170]:
rental_affordability_final

,year,rai
0,2011,1756.44
1,2012,1799.64
2,2013,1951.53
3,2014,1967.62
4,2015,1967.36
5,2016,2014.02
6,2017,2061.61
7,2018,2050.96
8,2019,2066.29
9,2020,2555.03


In [1]:
analysis_df = pd.merge(liveability_final, rental_affordability_final, how='inner')
analysis_df['year'] = analysis_df['year'].astype(int)
analysis_df

NameError: name 'pd' is not defined

Appears that all coefficients are not significant? Remove interaction and see which variable between year and rai has more impact on the liveability percentage